# Genome-wide identification and functional prediction of Pi-responsive lncRNAs in rice

Abstract: lncRNAs have been shown to play significant roles in stress response. However, no systematic screening of potential lncRNAs involved in Pi stress has been reported in plants, especially Oryza Sativa. In this study, we built the lncRNA identification pipeline and identified a total of 15,826 rice lncRNAs using 165 rice seedling RNA sequencing samples of Pi treatment. Through differential expression analysis with Cuffdiff, 1317 lncRNAs were termed as Pi-responsive lncRNAs as they are significantly up-regulated or down-regulated under Pi condition. To further explore Pi-responsive lncRNA functions, we performed miRNA target and target mimics analysis with psRobert and found some Pi-responsive lncRNAs were also targets and target mimics of miRNAs. With lncRNA expression level among samples, we performed lncRNA and gene co-expression analysis through WGCNA. It was found that some Pi-responsive lncRNAs co-expressed with neighboring or distant Pi-responsive protein coding genes. This study provides a comprehensive view of Pi-responsive lncRNAs, which will shed light on the regulatory mechanism involved in Pi response.

Pipeline: Firstly, we assembled the transcriptome with RNAseq datasets and identify lncRNAs with several selection processes. Secondly, we characterized lncRNAs in following items: (1) sequene characters, the distribution of their full length, exon number, GC content; (2) miRNA targets and target mimics; (3)DNA methylation pattern around lncRNA; (4) expression characters: expression level compared with protein coding genes, Pi responsive lncRNAs and co-expression with protein coding genes. Thirdly, we predict Pi-responsive lncRNA functions in Pi response: (1) they may down regulate miRNA expression as they are miRNA target mimics; (2) they may up or down regulate nearest Pi responsive gene expression through DNA methylation.

## Part One.  Identification of lncRNAs

### 1. Assemble the transcriptome with RNA-seq datasets.

```sh
bash /psc/bioinformatics/sunyd/identify/script/lncrna_prepare.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/assemble_gtf_listall.txt /psc/bioinformatics/sunyd/genome/oryza_sativa/osa_MSU7.gff3 /psc/bioinformatics/sunyd/genome/oryza_sativa/osa_MSU7.fa /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi 165
```

### 2. Identify lncRNAs with several selection processes.

```sh
for i in /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/cuff*.combinedhq.name.fa; do j=`echo $i |cut -d "/" -f1-7`; split -d -a 2 -l 2000 $i $j/split_cuff/cuff; cd $j/split_cuff; rename '0' '' cuff0*; done
qsub -t 1-70 /psc/bioinformatics/sunyd/identify/script/lncrnaml_qsuball_newtrain.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi
bash /psc/bioinformatics/sunyd/identify/script/lncrna_handle_rfmodel_pi.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi cuff165 /psc/bioinformatics/sunyd/genome/oryza_sativa/tRNAs_osa_GtRNAdb1.bed
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig1.png?raw=true" style="height: 400px;" />
Figure 1. A computational pipeline for the systematic identification of lncRNAs in this study. CPC: Coding Potential Calculator.

## Part Two. Characterization of lncRNAs

### 1. We characterized lncRNA sequence information, including length distribution, exon number distribution, GC content, and so on. Then we compared with protein coding genes to find its pattern.

```sh
bash ~/sunyd/identify/script/lncrna_sequence_info.sh ~/sunyd/identify/oryza_rnaseq/SRRpi
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig2.png?raw=true" style="height: 400px;" />
Figure 2. Characterization of lncRNAs. (B) Transcript size distributions for lncRNAs and protein-coding transcripts. (C) G/C content of lncRNAs and protein-coding transcripts. (D) The number of exons per transcript for all lncRNAs and protein-coding transcripts. (E) Exon size distributions for lincRNAs, lncNATs and protein-coding transcripts.

### 2. We analyzed DNA methlytion pattern around lncRNA and compared to protein coding genes.

```sh
bash /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/methylation.sh ~/sunyd/identify/oryza_rnaseq/SRRpi
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig3.png?raw=true" style="height: 500px;" />
Figure 3. DNA methylation in lncRNA and PCgene regions. For each gene, the upstream 1 kb, gene body and downstream 1 kb are characterized and divided into 100 bins, respectively.

### 3. We calculated lncRNA expression level among samples with cuffnorm and compared lncRNA expression level with protein coding genes.

```sh
bash /psc/bioinformatics/sunyd/identify/script/expression_gtf.sh cuff165 /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi /psc/bioinformatics/sunyd/genome/oryza_sativa/osa_MSU7
bash /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/cuffnorm.sh ~/sunyd/identify/oryza_rnaseq/SRRpi
bash /psc/bioinformatics/sunyd/identify/script/expression_level.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig4.png?raw=true" style="height: 300px;" />
Figure 4. The expression level of lncRNAs and PCgenes.

## Part Three.  Identification of Pi-responsive lncRNAs

### 1. We identified Pi differentially expressed lncRNAs with cuffdiff and cummerRbund.

```sh
bash /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/diff.sh
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig5.png?raw=true" style="height: 300px;" />
Figure 5. The number of differentially expressed lncRNAs under Pi condition.

### 2. We annotated the function of Pi differentially expressed lncRNAs with TopGO.

```sh
cd ~/sunyd/identify/oryza_rnaseq/SRRpi
grep -Fwf cuffdiff.028766_root/root.de.lncrna coexpression/go.correlation.txt |sed 's/\t/\n/g' |sort -u |sed '/TCONS/d' |awk 'NF=1' > coexpression/root.de.lncrna.correlation20
/psc/program/src/R-3.1.1/bin/Rscript ~/sunyd/identify/script/topgo.coexpression.hclus.R /psc/bioinformatics/sunyd/genome/oryza_sativa/osa_MSU7.go.csv.candidate ~/sunyd/identify/oryza_rnaseq/SRRpi/coexpression/root.de.lncrna.correlation20 ~/sunyd/identify/oryza_rnaseq/SRRpi/coexpression/root.de.lncrna20 38 0.05 0.0.5 0.05
Rscript ~/sunyd/identify/script/topgo.coexpression.hclus.plot.R root.de.lncrna20.upgo root.de.lncrna20     #######refine the picture
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig5_2.png?raw=true" style="height: 400px;" />
Figure 6. Functional annotation of differentially expressed lncRNAs.

## Part Four. Pi-responsive lncRNA functional prediction

### 1. lncRNA function as miRNA targets and target mimics.

#### 1) We identified mirna target and mimic using psRobert.

```sh
qsub -t 4-17 /psc/bioinformatics/sunyd/identify/script/psrobot.mimic.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/mirna/mirna.fa /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/cuff*-CPC_left.lncname.iuox.del.fa /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/mirna                                                                                          
bash /psc/bioinformatics/sunyd/identify/script/repeat_mirna.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi cuff165 rice /psc/bioinformatics/sunyd/genome/oryza_sativa/osa_mirna_miRbase.fa 
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig6.png?raw=true" style="height: 200px;" />
Figure 7. miRNA targets and target mimics. The top shows miRNA and its target; the middle one shows miRNA and its target mimic; the bottom one shows a regulatory role of osa-miR1435, which has both target and target mimic.

#### 2) We identified lncRNAs which are both Pi-responsive and miRNA target mimics.

```sh
cd ~/sunyd/identify/oryza_rnaseq/SRRpi/coexpression
grep -Fwf <(comm -1 -2 <(cut -f1 ../mirna/psrobot.mirna.mimic.name.de.lncrna |sort) <(cut -f1 gene.nearest.coexpressed.diff |sort)) gene.nearest.coexpressed.diff.all > gene.nearest.coexpressed.diff.all.mimic
grep -Fwf <(cut -f3 gene.nearest.coexpressed.diff.all.mimic |sort -u) ../mirna/psrobot.gene.target
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig7.png?raw=true" style="height: 300px;" />
Figure 8. lncRNA function as miRNA target mimic which corresponds to previous study.

### 2. lncRNA function through affecting Pi-responsive gene expression.

#### 1) We found lncRNA coexpressed genes with co-expression analysis. Based on expression level, we calculated Pearson correlation coefficient (Pcc) between lncRNA and gene through WGCNA.

```sh
wc /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi/cuffnorm.*/isoforms.fpkm_table
bash /psc/bioinformatics/sunyd/identify/script/coexpression.sh /psc/bioinformatics/sunyd/identify/oryza_rnaseq/SRRpi /psc/bioinformatics/sunyd/genome/oryza_sativa/osa_MSU7
```

#### 2) We identified lncRNAs which are both Pi-responsive and co-expressed with nearest Pi responsive genes.

```sh
bash ~/sunyd/identify/script/gene_nearest.sh ~/sunyd/identify/oryza_rnaseq/SRRpi
```

#### 3) We identified lncRNAs which are both Pi-responsive and co-expressed with experimental validated Pi-responsive genes.

```sh
bash ~/sunyd/identify/script/gene_vali_notnearest.sh ~/sunyd/identify/oryza_rnaseq/SRRpi
```

#### 4) We did qRT-PCR to validate lncRNA and co-expressed gene expression level.

```sh
bash ~/sunyd/identify/script/qrt-pcr.sh ~/sunyd/identify/oryza_rnaseq/SRRpi
```

<img src="https://github.com/fengkuangbaozha/TPLA/blob/fig/rice_fig8.png?raw=true" style="height: 600px;" />
Figure 9. The RNA-seq and qRT-PCR results of co-expression, which correspond to previous study. *: Pvalue< 0.05; **: Pvalue< 0.001.

### Notice: our partner is responsible for plant materials and qRT-PCR experimental validation.